In [9]:
import pandas as pd
import spacy
import os
import json
import operator
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
porter_stemmer = PorterStemmer()
import readability
import vaderSentiment
import numpy as np
import itertools 


#import sys 
#!{sys.executable} -m pip install vadersentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
from readability import Readability  #https://github.com/cdimascio/py-readability-metrics
stop_words = nltk.corpus.stopwords.words('english')
#from profanity_check import predict  # https://github.com/vzhou842/profanity-check
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
nlp = spacy.load('C://Users//seanp//Anaconda3//Lib//site-packages//en_core_web_sm//en_core_web_sm-2.0.0')


main_dict_path = os.getcwd()[:os.getcwd().find("feat_extraction")]

movies = pd.read_csv(main_dict_path + "success_data.csv", index_col=0)
movies["Success"] = movies.apply(lambda row: 1 if row["Worldwide ROI (%)"] > 0 else 0, axis=1)
movies.drop(movies.columns[1:7], axis=1, inplace=True)
movies.head()

,Processed Title,Success
Avatar,Avatar,1
The Dark Knight Rises,"Dark-Knight-Rises,-The",1
The Avengers,"Avengers,-The",1
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1
Men in Black 3,Men-in-Black-3,1


In [ ]:
path = main_dict_path + "diag_jsons/"
n_unique_words_dict = {"n_unique_words_char_1": [], "n_unique_words_char_2": [],
                      "n_unique_words_char_3": [], "n_unique_words_char_4": [],
                      "n_unique_words_char_5": []}
read_level_dict = {"FK_read_level_char_1": [], "FK_read_level_char_2": [],
                   "FK_read_level_char_3": [], "FK_read_level_char_4": [],
                   "FK_read_level_char_5": []}
n_stop_words = {"n_stop_words_char_1": [], "n_stop_words_char_2": [],
               "n_stop_words_char_3": [], "n_stop_words_char_4": [],
               "n_stop_words_char_5": []}
n_curse_words = {"n_curse_words_char_1": [], "n_curse_words_char_2": [],
               "n_curse_words_char_3": [], "n_curse_words_char_4": [],
               "n_curse_words_char_5": []}
overall_polarity = {"overall_polarity_char_1":[],"overall_polarity_char_2":[],
                 "overall_polarity_char_3":[],"overall_polarity_char_4":[],
                 "overall_polarity_char_5":[]}
num_pass_sents = {"num_pass_sents_char_1":[],"num_pass_sents_char_2":[],
                 "num_pass_sents_char_3":[],"num_pass_sents_char_4":[],
                 "num_pass_sents_char_5":[]}
num_sents = {"num_sents_char_1":[],"num_sents_char_2":[],
                 "num_sents_char_3":[],"num_sents_char_4":[],
                 "num_sents_char_5":[]}
n_coref_sents = {"n_coref_sents_char_1": [], "n_coref_sents_char_2": [],
                        "n_coref_sents_char_3": [], "n_coref_sents_char_4": [],
                        "n_coref_sents_char_5": []}

n_polarity_mentions = {"n_polarity_mentions_char_1": [], "n_polarity_mentions_char_2": [],
                        "n_polarity_mentions_char_3": [], "n_polarity_mentions_char_4": [],
                        "n_polarity_mentions_char_5": []}

polarity_of_mentions= {"polarity_of_mentions_char_1": [], "polarity_of_mentions_char_2": [],
                        "polarity_of_mentions_char_3": [], "polarity_of_mentions_char_4": [],
                        "polarity_of_mentions_char_5": []}


prob = []
for script in movies["Processed Title"]:
    ho_chars = {"ho_char_1": [],'ho_char_2' : [],'ho_char_3' : [], 'ho_char_4': [],'ho_char_5' : []}
    
    with open(path + script + "_script.json") as handle:
        char_diags = json.loads(handle.read())
        
    # Gets the dictionary for only the 5 characters with the most dialogue
    dict_lengths = {}
    # so we will create a list that will store all of the sentences?? 
    # or will it just store the scores? 
    # so if I want the polarity of all of the sentences referring to that character 
    # I could just build 
    # well, I could do % positive and % negative... but why not just build the thing 1st... yeah, ok. 
    for key, value in char_diags["dialogues"].items():
        a = value.replace("\n", "").replace(" ", "")
        b = value.replace("\n", "").replace(" ", " ")
        i = 1
        while True:
            before = len(a)
            a = a.replace("[" + str(i) + "]", "")
            i += 1
            if len(a) == before:
                break
        dict_lengths[key] = len(a)
    top_chars = [char[0] for char in sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    all_chars = list(char_diags["dialogues"].keys())
    # so I can run this and go over the top 5 characters in every movie...... 
    # or I THINK just get rid of the for char in all chars part... 
    for char in all_chars:
        if char not in top_chars:
            del char_diags["dialogues"][char]
    
    # Loops over the characters dialogues to extract some features (per character)
    n_char = 1
    for char, dialogue in char_diags["dialogues"].items():
        # Gets the dialogue in one long sentence and without [i]
        diag = dialogue.replace("\n", "")
        i = 1
        while True:
            before = len(diag)
            diag = diag.replace("[" + str(i) + "]", "")
            i += 1
            if len(diag) == before:
                break
        
        # FEAT #1: # Number of Unique words per character --> Numerical
        # Tokenizes the long sentence, actually, pedro I think it's still a string because the punctuation still exists!
        words = word_tokenize(diag)
        # segment the sentences 
        sentz = sentence_tokenizer.tokenize(diag)
        # iterate though each string inside of the list of sentences       
        num_passive = 0
        coref_sents = 0 
        for sentence in sentz:
            # the nlp was essentially using spacy's 'en' module on the sentence, which is commented out above 
            # because I had the worst time actually importing it from python without using the location of the module
            tagged_sent = nlp(sentence)
            # checking if any of the words in the sentence are tagged as being what spacy considers to be 'passive'
            passive_toks=[tok for tok in tagged_sent if (tok.dep_ == "nsubjpass" or tok.dep_ =="auxpass")]
            # this means we will add 1 to our variable that contains the number of passive sentences... 
            if passive_toks != []:
                num_passive +=1
            # this is to check if another character is referenced in that sentence 
            # which means it will actually count for AT LEAST one character being referenced 
            # because it will be binary, so we break the first time we see a character being referenced. 
            
            for charj in char_diags["dialogues"].keys():
                counter = 0
                if charj != char and charj: 
                    if diag.count(charj[0] + charj[1:].lower()) >0:
                        n_polarity_mentions["n_polarity_mentions_char_" + str(n_char)].append(sentence)
                        ho_chars['ho_char_' + str(n_char)].append(sentence)
                        
                        if counter <1:
                            coref_sents +=1
                    elif diag.count(charj) >0:
                        n_polarity_mentions["n_polarity_mentions_char_" + str(n_char)].append(sentence)
                        ho_chars['ho_char_' + str(n_char)].append(sentence)

                        if counter <1:
                            coref_sents +=1
                    elif diag.count(charj.lower()) >0:
                        if counter <1:
                            coref_sents +=1
                        n_polarity_mentions["n_polarity_mentions_char_" + str(n_char)].append(sentence)
                        ho_chars['ho_char_' + str(n_char)].append(sentence)



        # this gets the percent of passive sentences. 
         
        
        
        num_pass_sents["num_pass_sents_char_" + str(n_char)].append(num_passive)
        num_sents["num_sents_char_" + str(n_char)].append(len(sentz))
        n_coref_sents["n_coref_sents_char_"+ str(n_char)].append(coref_sents)
                    
        
        # Stems the tokens
        #punct_sings = ".!?,;:-_--\'\'\'``..."
        #words_stemmed = [porter_stemmer.stem(word) for word in words if word not in punct_sings]
        # Gets the number of unique stems
        #n_unique_words_dict["n_unique_words_char_" + str(n_char)].append(len(set(words_stemmed)))
        
        # FEAT #2: Flesch Kincaid Read Level per character --> Categorical
        # https://github.com/cdimascio/py-readability-metrics
        #r = Readability(diag)
        #read_level_dict["FK_read_level_char_" + str(n_char)].append(r.flesch_kincaid().grade_level)
        #prob.append(script)
        
        # FEAT #3: Number of Stop Words per character --> Numerical
        #n_stop_words["n_stop_words_char_" + str(n_char)].append(len(set([w for w in words if w.lower() in stop_words])))
        
        
        # FEAT #4: Number of curse words per character -> Numerical
        # https://github.com/vzhou842/profanity-check
        #n_curse = 0
        #for word in words:
        #    if predict([word]): 
        #        n_curse += 1
        #n_curse_words["n_curse_words_char_" + str(n_char)].append(n_curse)
        
        # Feat 5: Overall Polarity of Character's dialogue --> Numerical
        overall_polarity["overall_polarity_char_" + str(n_char)].append(analyser.polarity_scores(diag)['compound'])



        # I can easily break this into overall positive and overall negative as well..... 
        # and they are here, but just note... this takes A WHILE to run... 
        
        
        n_char += 1
        
    #polarity_of_mentions["polarity_of_mentions_char_" + '1'].append(analyser.polarity_scores(' '.join(ho_chars["ho_char_" + '1'])))
    #polarity_of_mentions["polarity_of_mentions_char_" + '2'].append(analyser.polarity_scores(' '.join(ho_chars["ho_char_" + '2'])))
    #polarity_of_mentions["polarity_of_mentions_char_" + '3'].append(analyser.polarity_scores(' '.join(ho_chars["ho_char_" + '3'])))
    #polarity_of_mentions["polarity_of_mentions_char_" + '4'].append(analyser.polarity_scores(' '.join(ho_chars["ho_char_" + '4'])))
    #polarity_of_mentions["polarity_of_mentions_char_" + '5'].append(analyser.polarity_scores(' '.join(ho_chars["ho_char_" + '5'])))

        

        
        
        


In [6]:
def append_feature(feature_dict):
    for key in feature_dict.keys():
        movies[key] = feature_dict[key]
        
append_feature(n_unique_words_dict)
append_feature(read_level_dict)
append_feature(n_stop_words)
append_feature(overall_polarity)
append_feature(num_pass_sents)
append_feature(num_sents)

# we will essentially get the total passive sentences words across ALLLL CHARACTERS for a movie 
# and divide by the total sentneces each of them speak 
for index, row in feat_frame.iterrows():
    



In [7]:
movies.head()

,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,FK_read_level_char_3,...,num_pass_sents_char_1,num_pass_sents_char_2,num_pass_sents_char_3,num_pass_sents_char_4,num_pass_sents_char_5,num_sents_char_1,num_sents_char_2,num_sents_char_3,num_sents_char_4,num_sents_char_5
Avatar,Avatar,1,670,565,252,425,276,2,2,2,...,13,8,5,2,3,428,227,127,132,74
The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,3,...,13,6,10,8,5,265,226,169,149,159
The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,3,...,11,7,3,4,3,286,319,122,133,55
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,1,...,13,6,6,5,3,331,207,134,132,73
Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,2,...,18,12,2,2,3,732,357,60,56,38


In [20]:
movies['tot_pass_sents'] = movies.num_pass_sents_char_1+movies.num_pass_sents_char_2+movies.num_pass_sents_char_3+movies.num_pass_sents_char_4+movies.num_pass_sents_char_5
    # ok so lets' try to extract something else!!! 
# 
movies['tot_sents'] = movies.num_sents_char_1+movies.num_sents_char_2+movies.num_sents_char_3+movies.num_sents_char_4+movies.num_sents_char_5
movies['passive_ratio'] = movies.tot_pass_sents/movies.tot_sents

In [21]:
movies.head()

,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,FK_read_level_char_3,...,num_pass_sents_char_4,num_pass_sents_char_5,num_sents_char_1,num_sents_char_2,num_sents_char_3,num_sents_char_4,num_sents_char_5,tot_sents,tot_pass_sents,passive_ratio
Avatar,Avatar,1,670,565,252,425,276,2,2,2,...,2,3,428,227,127,132,74,988,31,0.031377
The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,3,...,8,5,265,226,169,149,159,968,42,0.043388
The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,3,...,4,3,286,319,122,133,55,915,28,0.030601
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,1,...,5,3,331,207,134,132,73,877,33,0.037628
Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,2,...,2,3,732,357,60,56,38,1243,37,0.029767


In [24]:
movies = movies.drop(['num_pass_sents_char_1','num_pass_sents_char_2','num_pass_sents_char_3','num_pass_sents_char_4','num_pass_sents_char_5','num_sents_char_1','num_sents_char_2','num_sents_char_3','num_sents_char_4','num_sents_char_5'],axis=1)

In [25]:
movies.head()

,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,FK_read_level_char_3,...,n_stop_words_char_4,n_stop_words_char_5,overall_polarity_char_1,overall_polarity_char_2,overall_polarity_char_3,overall_polarity_char_4,overall_polarity_char_5,tot_sents,tot_pass_sents,passive_ratio
Avatar,Avatar,1,670,565,252,425,276,2,2,2,...,101,94,0.9919,0.9946,0.9861,0.7676,-0.9982,988,31,0.031377
The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,3,...,107,121,0.9871,-0.9975,0.8331,-0.9751,-0.9954,968,42,0.043388
The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,3,...,96,68,0.9704,0.9790,0.9189,-0.9782,0.9739,915,28,0.030601
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,1,...,103,86,0.9980,-0.9658,0.9959,0.9297,0.9926,877,33,0.037628
Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,2,...,79,67,0.9979,0.9839,0.9903,-0.9787,0.9617,1243,37,0.029767


In [26]:
movies.to_csv("movies_with_feats_sean.csv")

In [27]:
# try sentiment strength of all verbs??? 
# look for words that are 'content words' and then lk at teh length of the words by # of letters.. hah 

# number of similar setnences??? 
# cosine similarity of the dialogu of all characters??? 
# check if any of the top 5 charactes put positive content first. 
# percentage of 1st person pronouns v. names??? 

# so... do coreference resolution per sentence??? Yikes 
# avg counts of propositions per sentence?

# determine whether positive propositions are said 1st? 

# check if softener hedges or emphasizer hedges are used 

# polarity of sentences when another character is being mentioned... 


# for me.. checking if propositiosn are combined in sentences. i.e. looking for ration o sents with 1 to ones with 1+ 
# then how they are merged, with comma, with soemthing like also 

# we might want to check if there is ONE character 
# that gets mean things said about them more than like.... anyone else
# OOOOH OH OH... IS THERE A CHARACTER THAT IS NOT IN THE TOP 5 CHARACTERS 
# BUT THEY GET MENTIONED THE MOST?> 

In [28]:
for script in movies["Processed Title"].head():
    
    with open(path + script + "_script.json") as handle:
        char_diags = json.loads(handle.read())

In [34]:
char_diags

for dialogue, char in char_diags["dialogues"].items():
        # Gets the dialogue in one long sentence and without [i]
        dialogue = dialogue.split("\n")
        dialogue = [line for line in dialogue if line != '']
        #print(dialogue)
        
        tin_can = 0 
        for charj in chars_diags["dialogues"].keys():
            if charj != char and charj:
                n_other_char_mentions["n_mentions_others_char_" + str(n_char)][-1] += diag.count(charj[0] + charj[1:].lower())
                n_other_char_mentions["n_mentions_others_char_" + str(n_char)][-1] += diag.count(charj)
                n_other_char_mentions["n_mentions_others_char_" + str(n_char)][-1] += diag.count(charj.lower())
        
        


#  i = 1
#        while True:
#            before = len(diag)
#            diag = diag.replace("[" + str(i) + "]", "")
#            i += 1
#            if len(diag) == before:
 #               break

        n_char += 1

["[1]That? That nasty, greasy thing makes your life worth living? There's people eating here, younger than that duck. ", '[2]The tanks in the back, Wu. ', '[3]Hey, save the chop socky bullshit for the locals, Wu. We need to see the kitchen. ', "[4]Wow, I didn't think there was anything uglier than a catfish. Jay taps on the glass -- the fish SNARLS, making Jay JUMP. ", '[5]Look, Wu. You can serve it to your... out of town guests, but locals get earth fish. REAL earth fish. The kind that doesn\'t jump out of their stomach and get everyone asking a lot of annoying questions. Like, "Why is my lunch eating Daddy?" You got it?  DAMN, that\'s ugly. ', '[6]Something about this strike you as weird? ', "[7]Look at table two over there. They haven't ordered a thing. When's the last time Wu let someone sit at a table without ordering? And table five over there just drank his third cup of duck sauce. Now Kay's radar is up -- ", "[8]Because it's about to go down. Wu, at the door, closes the blinds.


['[1]I was talking about the noodles. Best noodles in town. And we are in ', '[2]Good evening, Mr. Wu. ', "[3]Not so fast, Wu. We'd like to take a look in the tanks. ", '[4]Jay, take a look at this. Wu protests, Jay blows by him. ', "[5]So this is what you're passing off as tuna? Reveal a giant tank holding a HUGE ALIEN FISH. ", '[6]Ugly AND a clear violation of Health ordinance 32, selling ', '[7]I guess it would be.  If you were Chinese! Reveal -- from the waist down, Wu has the BODY OF A SLUG. ', "[8]Aren't you forgetting something? ", '[9]Yeah, look -- clean fork. 10. ', '[10]Come to think of it... Why is Wu getting our noodles... outside? ', '[11]Then come get it, you worthless ', '[12]Oh, I know what happens next. 12. Kay draws his EQUALLY BADASS-LOOKING WEAPON -- ', "[13]I've been waiting forty years for another shot at you, scum. Kay takes aim and... CLICK! Empty. ", '[14]Damn! ', '[15]They deserved what they got. ', '[16]Will you settle for a drink? ', '[17]Yaz. Put him away 

["[1]When's the last time you conjugated anything? ", "[2]Thanks, darling. Hey could you cut this up for us? It's our anniversary. I'm romantic like that. A guard picks up the cake. Smirks and takes a dirty three- fingered scoop of frosting. ", "[3]I wouldn't do that. ", '[4]That\'s why. The aliens jump into Yaz\'s hands like a matching pair of grotesque ORGANIC SIDEARMS -- as vicious and bloodthirsty as their master, snarling and hungry for blood. The guards draw their weapons, but YAZ IS FASTER.-- he launches his aliens, taking them all out. Yaz "holsters" the aliens and... ', "[5]There's nothing sexier than a girl killing for me. She runs a hand over his bicep. ", "[6]I'm not just a man of brute force, you know. I prefer to be known for my rapier wit-- His tongue SHOOTS OUT OF HIS MOUTH -- like a rapier -- impaling the guard-- 5. ", '[7]--and tongue. The tongue retracts -- he and Devil Girl KISS. They collect weapons off the guards and use the aliens to saw open the door. ', "[8]You

['[1]Ready to bring the pain? Reveal Zed in his ridiculous bowling outfit. ', "[2]Where's Kay? We either roll or forfeit -- and I didn't put this on to forfeit! ", '[3]Well, must be a Tuesday. Okay, computer [twins]. Give me a trajectory and an ETA on those puppies. ', "[4]Damn. Welp, looks like we're all done for. Sorry, kid, it's been a good ride. ", '[5]Sure, great idea, if we had one. ', '[6]I wanted to build one years ago, but Kay talked me out of it. ', '[7]Detainees are on the tube to Toronto. No sign of Kay. Over. ']
["[1]Lookie here. Yaz's visitor. I guess even a turd gets flies to land on it. ", '[2]I.D., visitation papers. ', "[3]This ain't a conjugal visit. quit yer conjugating. ", "[4]Not great. She must.be good at somethin' else. They all LAUGH. He goes back for another scoop. ", "[5]Why's that, convict? 4. ", '[6]']


In [19]:
def cosine_similarity(document_1_data, document_2_data):
    document_vector_word_index = [] # here fill this with an ordered list of all the unique words across both documents
    d1_uniqs = []
    d1_nun = []
    dat = sentence_tokenizer.tokenize(document_1_data)
    for sent in dat:
        d1_uniqs.extend(word_tokenize(sent))
        d1_nun.extend(word_tokenize(sent))
        d1_uniqs = list(set(d1_uniqs))
    d2_uniqs = []
    d2_nun = []
    dat2 = sentence_tokenizer.tokenize(document_2_data)
    for sent in dat2:
        d2_uniqs.extend(word_tokenize(sent))
        d2_uniqs = list(set(d2_uniqs))
        d2_nun.extend(word_tokenize(sent))

    
    all_words = d1_nun+d2_nun   

    overlap = d1_uniqs + d2_uniqs 
    # so this is all of the unique words across the two documents
    document_vector_word_index += list(set(overlap))
    

    document_1_vector = list(np.zeros(len(document_vector_word_index)))  # fill in the array with the frequency of the words in the document
    document_2_vector = list(np.zeros(len(document_vector_word_index))) # fill in the array with the frequency of the words in the document

    for wOrd in d1_nun:
        document_1_vector[document_vector_word_index.index(wOrd)] +=1
    
    for wOrd in d2_nun:
        document_2_vector[document_vector_word_index.index(wOrd)] +=1
    
    d1_norm = (np.sum(np.array(document_1_vector)**2))**(1/2)
    d2_norm = (np.sum(np.array(document_2_vector)**2))**(1/2)
    
    cosSim = np.dot(document_1_vector,document_2_vector)/(d1_norm*d2_norm)


    
    return (cosSim)

In [228]:
cosine_similarity('Hello there, good sir! How are you?','Oh, Hello there, sir! How is life?')

['sir', ',', 'there', 'Hello', 'good', '!']
['?', 'sir', 'are', ',', 'you', 'How', 'there', 'Hello', 'good', '!']
['sir', 'Oh', ',', 'there', 'Hello', '!']
['Oh', ',', 'Hello', 'there', ',', 'sir', '!']
['?', 'is', 'sir', 'life', 'Oh', ',', 'How', 'there', 'Hello', '!']
['Oh', ',', 'Hello', 'there', ',', 'sir', '!', 'How', 'is', 'life', '?']


0.7016464154456235

In [234]:
import itertools 
cout = 0 
for i in itertools.combinations([1,2,3,4,5],2):
    sim_scores['sim_char_'+str(i[0]) + '_' + str(i[1])].append(cout)


In [280]:
polarity_of_mentions= {"polarity_of_mentions_char_1": [], "polarity_of_mentions_char_2": [],
                        "polarity_of_mentions_char_3": [], "polarity_of_mentions_char_4": [],
                        "polarity_of_mentions_char_5": []}

sim_scores = {'sim_char_1_2':[],'sim_char_1_3':[],'sim_char_1_4':[],
              'sim_char_1_5':[],'sim_char_2_3':[],'sim_char_2_4':[],
              'sim_char_2_5':[],'sim_char_3_4':[],'sim_char_3_5':[],
              'sim_char_4_5':[]}

In [235]:
sim_scores

{'sim_char_1_2': [1],
 'sim_char_1_3': [2],
 'sim_char_1_4': [3],
 'sim_char_1_5': [4],
 'sim_char_2_3': [5],
 'sim_char_2_4': [6],
 'sim_char_2_5': [7],
 'sim_char_3_4': [8],
 'sim_char_3_5': [9],
 'sim_char_4_5': [10]}

In [27]:
path = main_dict_path + "diag_jsons/"
prob = []
sim_scores = {'sim_char_1_2':[],'sim_char_1_3':[],'sim_char_1_4':[],
              'sim_char_1_5':[],'sim_char_2_3':[],'sim_char_2_4':[],
              'sim_char_2_5':[],'sim_char_3_4':[],'sim_char_3_5':[],
              'sim_char_4_5':[]}
timmay = 0 
for script in movies["Processed Title"]:
    #ho_chars = {"ho_char_1": [],'ho_char_2' : [],'ho_char_3' : [], 'ho_char_4': [],'ho_char_5' : []}
    clean_diags = {}
    timmay +=1
    if timmay == 15:
        print('we processed 15')
    if timmay == 50:
        print('50')
    if timmay == 200:
        print('TWO HUNRED!')
    with open(path + script + "_script.json") as handle:
        char_diags = json.loads(handle.read())
        
    # Gets the dictionary for only the 5 characters with the most dialogue
    dict_lengths = {}
    # so we will create a list that will store all of the sentences?? 
    # or will it just store the scores? 
    # so if I want the polarity of all of the sentences referring to that character 
    # I could just build 
    # well, I could do % positive and % negative... but why not just build the thing 1st... yeah, ok. 
    for key, value in char_diags["dialogues"].items():
        a = value.replace("\n", "").replace(" ", "")
        b = value.replace("\n", "").replace(" ", " ")
# so I want to iterate over all of the combinations of the characters
# so let's get a sorted list of the top 5 characters where the 1st index is the key of the main character (or one who speaks most)
# then we can get the cosine similarity scores of all of those characters
        #print(a)
        i = 1
        while True:
            before = len(b)
            a = a.replace("[" + str(i) + "]", "")
            b = b.replace("[" + str(i) + "]", "")
            i += 1
            if len(b) == before:
                break
        #clean_diags[key] = b
        dict_lengths[key] = len(b)
    top_chars = [char[0] for char in sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    all_chars = list(char_diags["dialogues"].keys())
    #all_chars_2 = list(clean_diags["dialogues"].keys())
    # so I can run this and go over the top 5 characters in every movie...... 
    # or I THINK just get rid of the for char in all chars part... 
    for char in all_chars:
        if char not in top_chars:
            del char_diags["dialogues"][char]
    
    
    
    # Loops over the characters dialogues to extract some features (per character)
    n_char = 1
    for char, dialogue in char_diags["dialogues"].items():
        # Gets the dialogue in one long sentence and without [i]
        diag = dialogue.replace("\n", " ")
        i = 1
        while True:
            before = len(diag)
            diag = diag.replace("[" + str(i) + "]", "")
            i += 1
            if len(diag) == before:
                break
        clean_diags[char] = diag
    
    import itertools 
    for i in itertools.combinations([0,1,2,3,4],2):
    
        sim_scores['sim_char_'+str(i[0]+1) + '_' + str(i[1]+1)].append(cosine_similarity(clean_diags[top_chars[i[0]]],clean_diags[top_chars[i[1]]]))
    
    
    n_char +=1

we processed 15
50
TWO HUNRED!


In [36]:
sim_scores['sim_char_1_2'].mean()

AttributeError: 'list' object has no attribute 'mean'

In [265]:
for i in clean_diags.keys():
    print(i)


WAYNE
BLAKE
BANE
GORDON
ALFRED


In [22]:
sim_scores = {'sim_char_1_2':[],'sim_char_1_3':[],'sim_char_1_4':[],
              'sim_char_1_5':[],'sim_char_2_3':[],'sim_char_2_4':[],
              'sim_char_2_5':[],'sim_char_3_4':[],'sim_char_3_5':[],
              'sim_char_4_5':[]}
import itertools 
for i in itertools.combinations([0,1,2,3,4],2):
    
    sim_scores['sim_char_'+str(i[0]+1) + '_' + str(i[1]+1)].append(cosine_similarity(clean_diags[top_chars[i[0]]],clean_diags[top_chars[i[1]]]))

In [23]:
sim_scores

{'sim_char_1_2': [0.9321511728780028],
 'sim_char_1_3': [0.9060310169229058],
 'sim_char_1_4': [0.9412108072660221],
 'sim_char_1_5': [0.9171031958801982],
 'sim_char_2_3': [0.8921085200772267],
 'sim_char_2_4': [0.9197421668167558],
 'sim_char_2_5': [0.9541152113578035],
 'sim_char_3_4': [0.9211398584733803],
 'sim_char_3_5': [0.9091929794650453],
 'sim_char_4_5': [0.9122295604136151]}

In [24]:
clean_diags[top_chars[i[0]]][0:20]

' Mr. Till, why are y'

In [37]:
def append_feature(feature_dict):
    for key in feature_dict.keys():
        movies[key] = feature_dict[key]
        
#append_feature(n_unique_words_dict)

In [40]:
append_feature(sim_scores)

In [41]:
movies

,Processed Title,Success,sim_char_1_2,sim_char_1_3,sim_char_1_4,sim_char_1_5,sim_char_2_3,sim_char_2_4,sim_char_2_5,sim_char_3_4,sim_char_3_5,sim_char_4_5
Avatar,Avatar,1,0.961107,0.944568,0.892316,0.932804,0.944773,0.887554,0.938676,0.885419,0.915689,0.852732
The Dark Knight Rises,"Dark-Knight-Rises,-The",1,0.932151,0.906031,0.941211,0.917103,0.892109,0.919742,0.954115,0.921140,0.909193,0.912230
The Avengers,"Avengers,-The",1,0.910721,0.928274,0.932986,0.861689,0.794249,0.879340,0.828118,0.905783,0.861437,0.878220
Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,0.917472,0.907316,0.917672,0.880575,0.895428,0.909860,0.824179,0.852723,0.816893,0.859171
Men in Black 3,Men-in-Black-3,1,0.976136,0.903336,0.878124,0.835116,0.895565,0.864982,0.843776,0.806134,0.787097,0.778844
King Kong,King-Kong,1,0.911559,0.867492,0.885605,0.899010,0.848877,0.852476,0.828081,0.806076,0.857503,0.860393
Guardians of the Galaxy Vol 2,Guardians-of-the-Galaxy-Vol-2,1,0.916857,0.958137,0.953772,0.941640,0.879613,0.881423,0.906311,0.940515,0.919964,0.929367
Cars 2,Cars-2,1,0.948520,0.901716,0.885196,0.862368,0.922384,0.910441,0.849520,0.955322,0.851294,0.844839
2012,2012,1,0.892760,0.893656,0.834770,0.887696,0.817694,0.890823,0.860160,0.807300,0.818436,0.770493
Terminator Salvation,Terminator-Salvation,1,0.879333,0.846445,0.893506,0.832343,0.883262,0.820580,0.888699,0.820037,0.868969,0.798825


In [44]:
mov1 = pd.read_csv("movies_with_feats_sean.csv")

In [45]:
mov1.head()

,Unnamed: 0,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,...,n_stop_words_char_4,n_stop_words_char_5,overall_polarity_char_1,overall_polarity_char_2,overall_polarity_char_3,overall_polarity_char_4,overall_polarity_char_5,tot_sents,tot_pass_sents,passive_ratio
0,Avatar,Avatar,1,670,565,252,425,276,2,2,...,101,94,0.9919,0.9946,0.9861,0.7676,-0.9982,988,31,0.031377
1,The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,...,107,121,0.9871,-0.9975,0.8331,-0.9751,-0.9954,968,42,0.043388
2,The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,...,96,68,0.9704,0.9790,0.9189,-0.9782,0.9739,915,28,0.030601
3,Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,...,103,86,0.9980,-0.9658,0.9959,0.9297,0.9926,877,33,0.037628
4,Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,...,79,67,0.9979,0.9839,0.9903,-0.9787,0.9617,1243,37,0.029767


In [46]:
mov2 = pd.read_csv("movies_with_feats_sean2.csv")

In [57]:
#mov2.drop(['polarity_of_mentions_char_1','polarity_of_mentions_char_2','polarity_of_mentions_char_3','polarity_of_mentions_char_4','polarity_of_mentions_char_5'],axis=1,inplace = True)
mov2.drop(['n_coref_sents_char_1','n_coref_sents_char_2','n_coref_sents_char_3','n_coref_sents_char_4','n_coref_sents_char_5'],axis=1,inplace = True)

In [59]:
mov3 = pd.read_csv("movies_with_feats_sean3.csv")

In [61]:
mov3.columns
mov3.drop(['polarity_of_mentions_char_1','polarity_of_mentions_char_2','polarity_of_mentions_char_3','polarity_of_mentions_char_4','polarity_of_mentions_char_5'],axis=1,inplace = True)


In [134]:
one23.head()

,Unnamed: 0,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,...,compound_polarity_of_mentions_char_3,compound_polarity_of_mentions_char_4,compound_polarity_of_mentions_char_5,tot_coref_sents,pct_coref_sents,n_coref_sents_char_1,n_coref_sents_char_2,n_coref_sents_char_3,n_coref_sents_char_4,n_coref_sents_char_5
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Avatar,Avatar,Avatar,1,670,565,252,425,276,2,2,...,0.9961,0.7676,-0.9982,54,0.054656,24,20,8,1,1
The Dark Knight Rises,The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,...,0.9404,-0.9966,-0.9987,56,0.057851,18,15,3,9,11
The Avengers,The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,...,0.9716,-0.9782,0.9929,60,0.065574,23,17,13,6,1
Pirates of the Caribbean: Dead Man's Chest,Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,...,0.0000,0.0000,0.0000,5,0.005701,4,1,0,0,0
Men in Black 3,Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,...,0.9987,-0.9939,0.9894,231,0.185841,137,70,9,9,6


In [89]:
#new_mov1 = mov1.loc['Processed Title', 'Success', 'n_unique_words_char_1','n_unique_words_char_2','n_unique_words_char_2','n_unique_words_char_3','n_unique_words_char_4','n_unique_words_char_5','FK_read_level_char_1', 'FK_read_level_char_2','FK_read_level_char_3', 'FK_read_level_char_4','FK_read_level_char_5', 'n_stop_words_char_1','n_stop_words_char_2','n_stop_words_char_3','n_stop_words_char_4','n_stop_words_char_5','overall_polarity_char_1','overall_polarity_char_2','overall_polarity_char_3''overall_polarity_char_4','overall_polarity_char_5','tot_sents','tot_pass_sents','passive_ratio']

In [130]:
one2 = mov1.join(mov2[oneNotin2])
one23 = one2.join(mov3[twoNotin3])

In [128]:
oneNotin2 = [i for i in mov2.columns if i not in mov1.columns]
twoNotin3 = [i for i in mov3.columns if i not in mov2.columns]

In [132]:
one23

,Unnamed: 0,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,...,compound_polarity_of_mentions_char_3,compound_polarity_of_mentions_char_4,compound_polarity_of_mentions_char_5,tot_coref_sents,pct_coref_sents,n_coref_sents_char_1,n_coref_sents_char_2,n_coref_sents_char_3,n_coref_sents_char_4,n_coref_sents_char_5
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Avatar,Avatar,Avatar,1,670,565,252,425,276,2,2,...,0.9961,0.7676,-0.9982,54,0.054656,24,20,8,1,1
The Dark Knight Rises,The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,506,418,434,2,2,...,0.9404,-0.9966,-0.9987,56,0.057851,18,15,3,9,11
The Avengers,The Avengers,"Avengers,-The",1,560,623,359,425,205,2,3,...,0.9716,-0.9782,0.9929,60,0.065574,23,17,13,6,1
Pirates of the Caribbean: Dead Man's Chest,Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,378,281,2,1,...,0.0000,0.0000,0.0000,5,0.005701,4,1,0,0,0
Men in Black 3,Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,...,0.9987,-0.9939,0.9894,231,0.185841,137,70,9,9,6
King Kong,King Kong,King-Kong,1,637,260,231,245,231,2,1,...,0.6024,-0.9984,-0.9982,64,0.096531,34,7,9,11,3
Guardians of the Galaxy Vol 2,Guardians of the Galaxy Vol 2,Guardians-of-the-Galaxy-Vol-2,1,589,463,348,480,318,2,2,...,-0.9989,0.9999,-0.9763,56,0.060150,21,16,12,4,3
Cars 2,Cars 2,Cars-2,1,688,389,410,376,232,2,1,...,0.9994,0.9992,0.0000,121,0.093149,16,53,30,22,0
2012,2012,2012,1,366,486,217,297,206,1,3,...,0.5819,-0.8251,0.9935,26,0.046181,8,6,6,1,5


In [111]:
oneNotin2[0]

'num_pass_sents_char_1'

In [119]:
one2.iloc[0,:]

Unnamed: 0                                 Avatar
Processed Title                            Avatar
Success                                         1
n_unique_words_char_1                         670
n_unique_words_char_2                         565
n_unique_words_char_3                         252
n_unique_words_char_4                         425
n_unique_words_char_5                         276
FK_read_level_char_1                            2
FK_read_level_char_2                            2
FK_read_level_char_3                            2
FK_read_level_char_4                            3
FK_read_level_char_5                            3
n_stop_words_char_1                           139
n_stop_words_char_2                           124
n_stop_words_char_3                            84
n_stop_words_char_4                           101
n_stop_words_char_5                            94
overall_polarity_char_1                    0.9919
overall_polarity_char_2                    0.9946


In [122]:
mov1.index = mov1['Unnamed: 0']

In [123]:
mov2.index = mov2['Unnamed: 0']

In [124]:
mov3.index = mov3['Unnamed: 0']

In [135]:
movies.to_csv("movies_with_feats_use_me.csv")